In [1]:
# Ref: https://qiita.com/noca/items/00646efd9d4a7302f522
from ipywidgets import Textarea
import io
if 'open' in globals():
    del open
if 'input' in globals():
    del input
original_open = open
class custom_open():
    def __init__(self):
        self.text = ''
    def __call__(self, file, *args, **kwargs):
        if file == 0:
            return io.StringIO(self.text)
        return original_open(file, *args, **kwargs)
    def updater(self, change):
        self.text = change["new"]
class custom_input():
    def __init__(self):
        self.__sio = io.StringIO('')
        self.shell = get_ipython()
        if self.shell.events.callbacks['pre_run_cell'] != []:
            self.shell.events.callbacks['pre_run_cell'] = []
        self.shell.events.register('pre_run_cell', self.pre_run_cell)
    def __call__(self):
        return self.__sio.readline().strip()
    def pre_run_cell(self, info):
        text = self.shell.user_ns.get('text_area', None).value
        self.__sio = io.StringIO(text)
open = custom_open()
input = custom_input()
text_area = Textarea()
text_area.observe(open.updater, names='value')
display(text_area)

Textarea(value='')

In [77]:
N, M = map(int, input().split())
P = list(map(int, input().split()))

class FordFulkerson:
    def __init__(self, N):
        self.N = N
        self.G = [[] for i in range(N)]
 
    def add_edge(self, fr, to, cap):
        forward = [to, cap, None]
        forward[2] = backward = [fr, 0, forward]
        self.G[fr].append(forward)
        self.G[to].append(backward)
 
    def add_multi_edge(self, v1, v2, cap1, cap2):
        edge1 = [v2, cap1, None]
        edge1[2] = edge2 = [v1, cap2, edge1]
        self.G[v1].append(edge1)
        self.G[v2].append(edge2)
 
    def dfs(self, v, t, f):
        if v == t:
            return f
        used = self.used
        used[v] = 1
        for e in self.G[v]:
            w, cap, rev = e
            if cap and not used[w]:
                d = self.dfs(w, t, min(f, cap))
                if d:
                    e[1] -= d
                    rev[1] += d
                    return d
        return 0
 
    def flow(self, s, t):
        flow = 0
        f = INF = 10 ** 30
        N = self.N 
        while f:
            self.used = [0] * N
            f = self.dfs(s, t, INF)
            flow += f
        return flow

f = FordFulkerson(N+2)
ans = 0
for i in range(N):
    if P[i] >= 0:
        ans += P[i]
        f.add_edge(i+1, N+1, P[i])
    else:
        f.add_edge(0, i+1, -1 * P[i])

for _ in range(M):
    a, b = map(int, input().split())
    f.add_edge(b, a, 10 ** 30)

print(ans - f.flow(0, N+1))

7
